# Домашнее задание к лекции «Функции и работа с данными»

## Задание 1

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [4]:
import pandas as pd

ratings = pd.read_csv('ml-latest-small/ratings.csv')

def get_rating(value):
    
    if value <= 2: 
        return 'низкий рейтинг'
    
    if value > 4:
        return 'высокий рейтинг'
    
    return 'средний рейтинг'

ratings['class'] = ratings['rating'].apply(get_rating)

ratings.head()

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,средний рейтинг
1,1,1029,3.0,1260759179,средний рейтинг
2,1,1061,3.0,1260759182,средний рейтинг
3,1,1129,2.0,1260759185,низкий рейтинг
4,1,1172,4.0,1260759205,средний рейтинг


### Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Результат классификации запишите в отдельный столбец region.

In [5]:
import re

keywords = pd.read_csv('ml-latest-small/keywords.csv')

def get_region(query):
    regions = [
        { 'cities': ['москва', 'тула', 'ярославль'], 'region': 'Центр' },
        { 'cities': ['петербург', 'псков', 'мурманск'], 'region': 'Северо-Запад' },
        { 'cities': ['владивосток', 'сахалин', 'хабаровск'] , 'region': 'Дальний Восток' },
    ]
    
    for region in regions:
        for city in region['cities']:
            
            if re.search(rf'{city}', query, flags=re.I):
                return region['region']
            
    return 'undefined'
            
        
keywords['region'] = keywords['keyword'].apply(get_region)

keywords['region'].value_counts().head()

undefined         99253
Центр               387
Северо-Запад        266
Дальний Восток       94
Name: region, dtype: int64

### Задание 3
Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

для каждой строки пройдите по всем годам списка years
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [6]:
years = list(range(1950, 2011))

ratings = pd.read_csv('ml-latest-small/ratings.csv')
movies = pd.read_csv('ml-25m/movies.csv')

res = ratings.merge(movies, on='movieId', how='left')

def production_year(movie):
    
    for year in years:  
        
        if str(year) in str(movie):   
            return year
        
    return 1900


res['year'] = res['title'].apply(production_year)

res.groupby('year').mean()[['rating']].reset_index().sort_values('rating', ascending=False).head()

,year,rating
8,1957,4.014241
23,1972,4.011136
3,1952,4.000000
5,1954,3.992754
2,1951,3.983539
